<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
# TODO - Words, words, mere words, no matter from the heart.

In [79]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import requests

In [80]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding

data = r.text
data = data.split('\r\n')

data = data[135:]

sonets = data[:2776]
plays = data[2777:]

In [81]:
def long_lines(lst_ln):
    clean = []
    
    for ln in lst_ln:
        
        if len(ln) == 0:
            pass
        
        else:
            pct = len(ln.strip(' ')) / len(ln)
            
            if pct >= .5:
                clean.append(ln.lstrip())
                
    return clean

In [82]:
sonets = long_lines(sonets)
plays = long_lines(plays)

In [83]:
## word encodings

vocab = list(set("\r\n".join(plays).split()))
words = [line.split() for line in plays]

In [84]:
len(vocab)

75585

In [85]:
#character encoding

text = '\r\n'.join(plays)
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

len(chars)

106

In [86]:


maxlen = 100

step = 5

encoded = [char_int[c] for c in text]

sequences = [] # each element is 40 char ling
next_char = [] # one element fo reach sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1064477


In [87]:
# create x and y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    
    y[i, next_char[i]] = 1

In [88]:
x.shape

(1064477, 100, 106)

In [89]:
# build the model: a single LSTM


model = Sequential()
model.add(LSTM(1, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adagrad')

In [90]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1)                 432       
_________________________________________________________________
dense_5 (Dense)              (None, 106)               212       
Total params: 644
Trainable params: 644
Non-trainable params: 0
_________________________________________________________________


In [94]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [95]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=1.0)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [96]:
# fit the model
model.fit(x,y,
         batch_size=512,
         epochs=20,
         callbacks=[print_callback])

Epoch 1/20
1064448/1064477 [============================>.] - ETA: 0s - loss: 3.4484
----- Generating text after Epoch: 0
----- Generating with seed: "ight-gown in respect of yours: cloth o’
gold, and cuts, and laced with silver, set with pearls, dow"
ight-gown in respect of yours: cloth o’
 old, and cuts, and laced with silver, set with pearls, dowersaoe 
,ta .ows cuedLEnil,ArreUtruhroa"srigUhi aBusoaàtcuRnS’ei tusamrwaKukaTsneayeeianamoc ongt p sd I umve9” c
nretoADH  
oReto-@ii. traisYoiywt[tawa e’nmîtehi.qewcEId osn tgtleiedesi_rhrrye  lœet ns:ioog e ao nnirifaeo
]M ’ 8lînaYtf Vthbu,  O
,xgete elw rsiEesXca nnooeTa r3
3o
Ino:nres,N AnhdGyiM%Ls@sgyomohfo a f.rel  ro.o e
bNbOLHsnhou la8tnnfg tn’ tft onwœ:liYut
1064477/1064477 [==============================] - 203s 191us/sample - loss: 3.4484
Epoch 2/20
1064448/1064477 [============================>.] - ETA: 0s - loss: 3.3796
----- Generating text after Epoch: 1
----- Generating with seed: "rd, I aim a mile beyond the moon;
Your lett

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN